In [1]:
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import pickle

/home/harshit/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [28]:
## Finding Mean HQ and ER returns

state = pd.read_csv("/home/harshit/geography/district.csv")
state = list(set(state["STATE"]))
state.sort()
stateer = pd.read_csv("/home/harshit/geography/ercoffcompanies.csv")
#stateer=stateer.replace("\+ACY-","&",regex=True)
#hq_path = "/home/harshit/geography/hqportfolios/"
hq_path = "/home/harshit/geography/hqcoffportfolios/"

for h in state:
    date = pd.DataFrame(columns=['Month'])
    month = pd.DataFrame(columns=['Month'])
    if(os.path.exists(hq_path+h+".csv")==True):
        hqread = pd.read_csv(hq_path+h+".csv")
        date = pd.DataFrame(columns=['Month'])
        for j in hqread["Company"]:
            j=j.upper()
            df = pd.read_csv("/home/harshit/geography/mret/"+j+".csv")
            df.rename(columns={"Return":j+"HQ"},inplace=True)
            date = pd.merge(date,df,left_on="Month", right_on="Month",how="outer",sort=False)
        date["meanHQ"] = round(np.mean(date,axis=1),2)
        date = date[:-3]
    
    if(h in stateer):
        comp=stateer[h][pd.notna(stateer[h])]
        month = pd.DataFrame(date["Month"])
        for i in comp:
            i=i.upper()
            ret = pd.read_csv("/home/harshit/geography/mret/"+i[0:-2]+".csv")
            ret.rename(columns={"Return":i+"ER"},inplace=True)

            if "14" in i:
                if("February-2014" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="February-2014"].index.values[0]:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2014"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "15" in i:
                if("January-2015" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2015"].index.values[0]:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2015"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")

            if "16" in i:
                if("January-2016" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2016"].index.values[0]:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2016"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "17" in i:
                if("January-2017" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2017"].index.values[0]:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2017"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")


            if "18" in i:
                if("January-2018" in ret.Month.values):
                    ret=ret[ret.loc[ret["Month"]=="January-2018"].index.values[0]:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")
                else:
                    ret=ret[:ret.loc[ret["Month"]=="December-2018"].index.values[0]+1]
                    month=pd.merge(month,ret,on="Month",how="outer")        

        
        month["meanER"] = round(np.mean(month,axis=1),2)
        
        
    date = pd.merge(date,month,left_on="Month",right_on="Month",how="outer",sort=False)  
    if(len(date!=0)):
        date.to_csv("/home/harshit/geography/coffmeanret/"+h+".csv",index=False)

In [46]:
factors = pd.read_csv("/home/harshit/geography/fourfactorssurvivorshipbiasadjusted.csv")

In [49]:
#Making model to preditct meanER return using meanHQ return as independent variable

lis = os.listdir("/home/harshit/geography/coffmeanret/")
for i in lis:
    data = pd.read_csv("/home/harshit/geography/coffmeanret/"+i)
    if ("meanER" in data and "meanHQ" in data):
        merged = pd.merge(factors,data[["meanHQ","meanER"]],left_index=True,right_index=True)
        merged=merged.dropna(axis=0,how='any')
        merged["meanHQ"]=merged["meanHQ"]-merged["Rf"]
        merged["meanER"]=merged["meanER"]-merged["Rf"]
        X= merged[["SMB","HML","WML","Rm-Rf","meanER"]]
        X = sm.add_constant(X)
        y = merged["meanHQ"]
        model1= sm.OLS(y,X,hasconst=True).fit()
        model1_name = "/home/harshit/geography/modelssbacoff/PredHQ/withER/"+i+".sav" #surviorship bias adjusted four factors
        pickle.dump(model1,open(model1_name,"wb"))                                  #(hqcoff)-models use corporate office as HQ state
        
        me = merged.drop(["Month"],axis=1)
        corr = me.corr()
        heat=corr.style.background_gradient(cmap='coolwarm').set_precision(2)
        print("\n"+i)
        display(heat)


GUJARAT.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,0.059,0.054
HML,0.16,1,-0.41,0.58,3.6e-05,0.58,0.28,0.42
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,-0.033,-0.19
Rm,0.16,0.58,-0.23,1,0.22,1,0.56,0.9
Rf,0.34,3.6e-05,-0.17,0.22,1,0.2,0.25,0.21
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,0.56,0.9
meanHQ,0.059,0.28,-0.033,0.56,0.25,0.56,1,0.39
meanER,0.054,0.42,-0.19,0.9,0.21,0.9,0.39,1



DELHI.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,0.083,0.017
HML,0.16,1,-0.41,0.58,3.6e-05,0.58,0.57,0.44
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,-0.23,-0.27
Rm,0.16,0.58,-0.23,1,0.22,1,0.89,0.88
Rf,0.34,3.6e-05,-0.17,0.22,1,0.2,0.19,0.26
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,0.89,0.87
meanHQ,0.083,0.57,-0.23,0.89,0.19,0.89,1,0.82
meanER,0.017,0.44,-0.27,0.88,0.26,0.87,0.82,1



KARNATAKA.csv


...

,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,-0.15,-0.014
HML,0.16,1,-0.41,0.58,3.6e-05,0.58,0.45,0.48
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,-0.36,-0.24
Rm,0.16,0.58,-0.23,1,0.22,1,0.48,0.91
Rf,0.34,3.6e-05,-0.17,0.22,1,0.2,-0.031,0.22
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,0.49,0.91
meanHQ,-0.15,0.45,-0.36,0.48,-0.031,0.49,1,0.47
meanER,-0.014,0.48,-0.24,0.91,0.22,0.91,0.47,1



WEST BENGAL.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,0.059,-0.16
HML,0.16,1,-0.41,0.58,3.6e-05,0.58,0.35,0.36
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,-0.38,-0.15
Rm,0.16,0.58,-0.23,1,0.22,1,0.77,0.44
Rf,0.34,3.6e-05,-0.17,0.22,1,0.2,0.35,-0.13
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,0.77,0.44
meanHQ,0.059,0.35,-0.38,0.77,0.35,0.77,1,0.31
meanER,-0.16,0.36,-0.15,0.44,-0.13,0.44,0.31,1



MAHARASHTRA.csv


,SMB,HML,WML,Rm,Rf,Rm-Rf,meanHQ,meanER
SMB,1,0.16,0.039,0.16,0.34,0.16,-0.017,0.051
HML,0.16,1,-0.41,0.58,3.6e-05,0.58,0.46,0.43
WML,0.039,-0.41,1,-0.23,-0.17,-0.23,-0.24,-0.11
Rm,0.16,0.58,-0.23,1,0.22,1,0.9,0.77
Rf,0.34,3.6e-05,-0.17,0.22,1,0.2,0.15,0.25
Rm-Rf,0.16,0.58,-0.23,1,0.2,1,0.9,0.77
meanHQ,-0.017,0.46,-0.24,0.9,0.15,0.9,1,0.64
meanER,0.051,0.43,-0.11,0.77,0.25,0.77,0.64,1


In [35]:
i

'JAMMU & KASHMIR.csv'

In [51]:
#Making model to preditct meanER return without meanHQ return as independent variable
lis = os.listdir("/home/harshit/geography/coffmeanret/")
for i in lis:
    data = pd.read_csv("/home/harshit/geography/coffmeanret/"+i)
    if ("meanHQ" in data):
        merged = pd.merge(factors,data[["meanHQ"]],left_index=True,right_index=True)
        merged=merged.dropna(axis=0,how='any')
        merged["meanHQ"]=merged["meanHQ"]-merged["Rf"]
        X= merged[["SMB","HML","WML","Rm-Rf"]]
        X = sm.add_constant(X)
        y = merged["meanHQ"]
        model= sm.OLS(y,X,hasconst=True).fit()   
        model_name = "/home/harshit/geography/modelssbacoff/PredHQ/withoutER/"+i+".sav" #surviorship bias adjusted four factors
        pickle.dump(model,open(model_name,"wb"))

In [56]:
# Getting Summary of meanER prediction models
for i in os.listdir("/home/harshit/geography/models/PredER/withHQ"):
    model1 = pickle.load(open("/home/harshit/geography/models/PredER/withHQ/"+i,"rb"))
    model2 = pickle.load(open("/home/harshit/geography/models/PredER/withoutHQ/"+i,"rb"))
    print(i.split(".")[0])
    print("\nSummary for prediction of meanER with meanHQ\n")
    print(model1.summary())
    print("Summary for prediction of meanER without meanHQ\n")
    print(model2.summary())

MAHARASHTRA

Summary for prediction of meanER with meanHQ

                            OLS Regression Results                            
Dep. Variable:                 meanER   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.682
Method:                 Least Squares   F-statistic:                     25.92
Date:                Wed, 17 Apr 2019   Prob (F-statistic):           3.89e-13
Time:                        19:16:54   Log-Likelihood:                -143.31
No. Observations:                  59   AIC:                             298.6
Df Residuals:                      53   BIC:                             311.1
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------